In [1]:
import datetime as dt
import numpy as np
import sigmf
from sigmf import SigMFFile, sigmffile
from sigmf.utils import get_data_type_str

# Rules on sigmf data formats
numpy will save I and Q data as a+bi, but sigmf would prefer they were separate arrays. Fix this by vectorizing a complex array as x.real and x.imag

Complex samples MUST be interleaved, with the in-phase component first (i.e., I[0] Q[0] I[1] Q[1] ... I[n] Q[n]). When core:num_channels in the Global object (described below) indicates that the Recording contains more than one channel, samples from those channels MUST be interleaved in the same manner, with the same index from each channel's sample serially in the Recording. For example, a Recording with two channels of ri16_le representing real-valued audio data from a stereo Recording and here labeled L for left and R for right, the data MUST appear as L[0] R[0] L[1] R[1] ... L[n] R[n]. The data type specified by core:data_type applies to all channels of data both real and imaginary parts.

# Rules for all files:

All filetypes MUST be stored in separate files on-disk.
It is RECOMMENDED that filenames use hyphens to separate words rather than whitespace or underscores.
Rules for SigMF Metadata files:

A Metadata file MUST only describe one Dataset file.
A Metadata file MUST be stored in UTF-8 encoding.
A Metadata file MUST have a .sigmf-meta filename extension.
A Metadata file MUST be in the same directory as the Dataset file it describes.
It is RECOMMENDED that the base filenames (not including file extension) of a Recording's Metadata and Dataset files be identical.
Rules for SigMF Dataset files:

The Dataset file MUST have a .sigmf-data filename extension.
Rules for SigMF Non-Conforming Dataset files:

The NCD file MUST NOT have a .sigmf-data filename extension.
Rules for SigMF Collection files:

The Collection file MUST be stored in UTF-8 encoding.
The Collection file MUST have a .sigmf-collection filename extension.
The sigmf-collection file MUST be either in the same directory as the Recordings that it references, or in the top-level directory of an Archive (described in later section).
Rules for SigMF Archive files:

The Archive MUST use the tar archive format, as specified by POSIX.1-2001.
The Archive file's filename extension MUST be .sigmf.
The Archive MUST contain the following files: for each contained Recording with some name given here meta-syntactically as N, files named N (a directory), N/N.sigmf-meta, and N/N.sigmf-data.
The Archive MAY contain a .sigmf-collection file in the top-level directory.
It is RECOMMENDED that if Recordings in an archive represent continuous data that has been split into separate Recordings, that their filenames reflect the order of the series by appending a hyphenated zero-based index (e.g., N-0, N-1, N-2, etc.,).

In [5]:
#get_data_type_str only support numpy native datatypes?

# suppose we have an complex timeseries signal
# data = np.zeros(1024, dtype=np.csingle)
A = 127
fs = 100e6
fc = 915e6
pw = 10e-6
cr = 5e6/pw
t = np.arange(0, pw, 1/fs)
testSignal = (A*np.exp(1j*np.pi*cr*t*t)*np.exp(1j*2*np.pi*fc*t))
data = np.dstack((testSignal.real,testSignal.imag)).reshape(testSignal.real.shape[0],-1).astype('float32')

# write those samples to file in cf32_le
data.tofile('./chirp-pulse.sigmf-data')

# create the metadata
meta = SigMFFile(
    data_file='./chirp-pulse.sigmf-data', # extension is optional
    global_info = {
        SigMFFile.DATATYPE_KEY: get_data_type_str(data),  # in this case, 'cf32_le'
        SigMFFile.SAMPLE_RATE_KEY: fs,
        SigMFFile.AUTHOR_KEY: 'cgzoghby@gmail.com',
        SigMFFile.DESCRIPTION_KEY: 'Example chirping signal file.',
        SigMFFile.VERSION_KEY: sigmf.__version__,
    }
)

# create a capture key at time index 0
meta.add_capture(0, metadata={
    SigMFFile.FREQUENCY_KEY: fc,
    SigMFFile.DATETIME_KEY: dt.datetime.utcnow().isoformat()+'Z',
})

# add an annotation at sample 100 with length 200 & 10 KHz width
meta.add_annotation(100, 200, metadata = {
    SigMFFile.FLO_KEY: 914995000.0,
    SigMFFile.FHI_KEY: 915005000.0,
    SigMFFile.COMMENT_KEY: 'example annotation',
})

# check for mistakes & write to disk
assert meta.validate()
meta.tofile('./chirp-pulse.sigmf-meta') # extension is optional

In [6]:
# Load a dataset
filename = './chirp-pulse.sigmf-data' # extension is optional
signal = sigmffile.fromfile(filename)

# Get some metadata and all annotations
sample_rate = signal.get_global_field(SigMFFile.SAMPLE_RATE_KEY)
sample_count = signal.sample_count
signal_duration = sample_count / sample_rate # Always in seconds?
annotations = signal.get_annotations()
data = signal.read_samples()

# Iterate over annotations
for adx, annotation in enumerate(annotations):
    annotation_start_idx = annotation[SigMFFile.START_INDEX_KEY]
    annotation_length = annotation[SigMFFile.LENGTH_INDEX_KEY]
    annotation_comment = annotation.get(SigMFFile.COMMENT_KEY, "[annotation {}]".format(adx))

    # Get capture info associated with the start of annotation
    capture = signal.get_capture_info(annotation_start_idx)
    freq_center = capture.get(SigMFFile.FREQUENCY_KEY, 0)
    freq_min = freq_center - 0.5*sample_rate
    freq_max = freq_center + 0.5*sample_rate

    # Get frequency edges of annotation (default to edges of capture)
    freq_start = annotation.get(SigMFFile.FLO_KEY)
    freq_stop = annotation.get(SigMFFile.FHI_KEY)

    # Get the samples corresponding to annotation
    samples = signal.read_samples(annotation_start_idx, annotation_length)

In [7]:
signal

SigMFFile({
    "global": {
        "core:author": "cgzoghby@gmail.com",
        "core:datatype": "f32_le",
        "core:description": "Example chirping signal file.",
        "core:sample_rate": 100000000.0,
        "core:sha512": "88993e3b7c1c543c07bf049811ec199e1b6f856758fa82189e6e99c310f4c611f6437bb17071d397ec6c052bed18c202ebcafeced90cf0d274211820f67c3a53",
        "core:version": "1.0.0"
    },
    "captures": [
        {
            "core:datetime": "2022-07-20T10:23:50.361198Z",
            "core:frequency": 915000000.0,
            "core:sample_start": 0
        }
    ],
    "annotations": [
        {
            "core:comment": "example annotation",
            "core:freq_lower_edge": 914995000.0,
            "core:freq_upper_edge": 915005000.0,
            "core:sample_count": 200,
            "core:sample_start": 100
        }
    ]
})

In [8]:
data = signal.read_samples()

In [10]:
data.shape,testSignal.shape

((2002,), (1001,))